In [1]:
#balancing (for sandra) 
library(themis)

ups_recipe <- recipe(Class ~ ., data = rare_cancer) |>
  step_upsample(Class, over_ratio = 1, skip = FALSE) |>
  prep()

ups_recipe

ERROR: Error in library(themis): there is no package called ‘themis’


"hello"